In [1]:
import pandas as pd
import shapefile
import numpy as np

In [2]:
holc = shapefile.Reader(r"..\..\Local\Tracts_2020_HOLC\Tracts_2020_HOLC.shp")
county_list = pd.read_excel("..\Public Data\VARIABLE for coder X.xlsx")
disaster = pd.read_csv("..\..\Local\DisasterDeclarationsSummaries.csv")

In [3]:
holc_df=pd.DataFrame([holc.record(i).as_dict() for i in range(len(holc))])
holc_df["COUNTYGISJOIN"] = holc_df["GISJOIN"].apply(lambda x: x[1:8])

In [4]:
holc_df[holc_df["MAX_state"]=="NY"]

,GISJOIN,neighborho,SUM_Perc,FIRST_holc,MAX_state,MAX_city,Shape_Leng,Shape_Area,COUNTYGISJOIN
24383,G3600010000100,6583,0.140164,C,NY,Albany,2544.168894,3.661746e+05,3600010
24384,G3600010000100,6584,0.043440,D,NY,Albany,1393.869508,1.134861e+05,3600010
24385,G3600010000201,6585,0.967765,D,NY,Albany,3550.040580,5.960161e+05,3600010
24386,G3600010000202,6585,0.405793,D,NY,Albany,3420.783006,5.849625e+05,3600010
24387,G3600010000202,6586,0.009093,D,NY,Albany,715.218398,1.310691e+04,3600010
...,...,...,...,...,...,...,...,...,...
30442,G3601190010702,7216,0.022953,A,NY,Lower Westchester Co.,2499.490166,7.417914e+04,3601190
30443,G3601190015100,7190,0.015984,B,NY,Lower Westchester Co.,1644.702847,3.697738e+04,3601190
30444,G3601190015100,7203,0.060102,B,NY,Lower Westchester Co.,1995.416912,1.390446e+05,3601190
30445,G3601190015100,7204,0.033437,B,NY,Lower Westchester Co.,1667.952330,7.735543e+04,3601190


In [5]:
holc_df=holc_df[holc_df["FIRST_holc"].isin(["C","D","E"])]
holc_df

,GISJOIN,neighborho,SUM_Perc,FIRST_holc,MAX_state,MAX_city,Shape_Leng,Shape_Area,COUNTYGISJOIN
0,G0100730000100,180,0.092969,D,AL,Birmingham,3336.386272,7.095820e+05,0100730
1,G0100730000100,181,0.161693,D,AL,Birmingham,7731.380924,1.234060e+06,0100730
2,G0100730000100,182,0.170326,D,AL,Birmingham,6111.241095,1.300062e+06,0100730
3,G0100730000100,185,0.258259,C,AL,Birmingham,7905.509799,1.971179e+06,0100730
5,G0100730000300,182,0.839850,D,AL,Birmingham,6759.740567,1.745631e+06,0100730
...,...,...,...,...,...,...,...,...,...
42069,G5501390001400,5335,0.025514,D,WI,Oshkosh,1604.035917,1.607568e+05,5501390
42070,G5501390001500,5331,0.101925,C,WI,Oshkosh,3580.024409,5.770248e+05,5501390
42071,G5501390001500,5332,0.019389,C,WI,Oshkosh,1719.647010,1.097678e+05,5501390
42072,G5501390001500,5333,0.074008,D,WI,Oshkosh,3105.756666,4.189811e+05,5501390


In [6]:
def fips_to_gisjoin(statex,countyx):
    statex = str(statex)
    countyx = str(countyx)
    if len(statex) == 1:
        statex = "0"+statex
    if len(countyx)==1:
        countyx = "0000"+countyx
    elif len(countyx) == 2:
        countyx = "000"+countyx
    elif len(countyx)==3:
        countyx = "00"+countyx
    elif len(countyx)==4:
        countyx = "0"+countyx
    result = statex + countyx
    if len(result)!=7:
        print(statex,countyx)
    assert len(result)==7

    return result


In [7]:
county_list["converted_gisjoin"]=county_list.apply(lambda x: fips_to_gisjoin(x["STATEFIPS"],x["COUNTYFIPS"]),axis=1)

In [8]:
def FIPS_transformer(sx,x):
    if x % 10 == 0 and sx not in ["02","15"]:
        x = str(int(x/10))
    else:
        if str(sx) == "15":
            x = str(x)[0]
        x = int(x)

    return int(x)

In [9]:
county_list["FIPS_remove0"]=county_list.apply(lambda x:FIPS_transformer(x["STATEFIPS"],x["COUNTYFIPS"]),axis=1)
county_list

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,converted_gisjoin,FIPS_remove0
0,Alabama,G0100010,41.0,1,10,Autauga,NaN,0100010,1
1,Alabama,G0100030,41.0,1,30,Baldwin,NaN,0100030,3
2,Alabama,G0100050,41.0,1,50,Barbour,NaN,0100050,5
3,Alabama,G0100070,41.0,1,70,Bibb,NaN,0100070,7
4,Alabama,G0100090,41.0,1,90,Blount,NaN,0100090,9
...,...,...,...,...,...,...,...,...,...
3253,Wyoming,G5500650,68.0,56,390,Teton,NaN,5600390,39
3254,Wyoming,G5500670,68.0,56,410,Uinta,NaN,5600410,41
3255,Wyoming,G5500690,68.0,56,430,Washakie,NaN,5600430,43
3256,Wyoming,G5500710,68.0,56,450,Weston,NaN,5600450,45


In [10]:
disaster_df = disaster[["femaDeclarationString","declarationType","declarationDate","incidentType","fipsStateCode","fipsCountyCode"]]
disaster_df=disaster_df.groupby(["fipsStateCode","fipsCountyCode"]).agg({"declarationType":"count"}).reset_index(drop=False)
disaster_df

,fipsStateCode,fipsCountyCode,declarationType
0,1,0,11
1,1,1,23
2,1,3,37
3,1,5,27
4,1,7,19
...,...,...,...
3319,72,153,30
3320,78,0,3
3321,78,10,27
3322,78,20,25


In [11]:
disaster_df[(disaster_df["fipsCountyCode"]==1)&(disaster_df["fipsStateCode"]==1)]

,fipsStateCode,fipsCountyCode,declarationType
1,1,1,23


In [12]:
county_list[(county_list["STATEFIPS"]==1)&(county_list["FIPS_remove0"]==1)]

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,converted_gisjoin,FIPS_remove0
0,Alabama,G0100010,41.0,1,10,Autauga,NaN,0100010,1


In [13]:
county_list.iloc[0,-1]

1

In [14]:
county_list

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,converted_gisjoin,FIPS_remove0
0,Alabama,G0100010,41.0,1,10,Autauga,NaN,0100010,1
1,Alabama,G0100030,41.0,1,30,Baldwin,NaN,0100030,3
2,Alabama,G0100050,41.0,1,50,Barbour,NaN,0100050,5
3,Alabama,G0100070,41.0,1,70,Bibb,NaN,0100070,7
4,Alabama,G0100090,41.0,1,90,Blount,NaN,0100090,9
...,...,...,...,...,...,...,...,...,...
3253,Wyoming,G5500650,68.0,56,390,Teton,NaN,5600390,39
3254,Wyoming,G5500670,68.0,56,410,Uinta,NaN,5600410,41
3255,Wyoming,G5500690,68.0,56,430,Washakie,NaN,5600430,43
3256,Wyoming,G5500710,68.0,56,450,Weston,NaN,5600450,45


In [15]:
combined = county_list.merge(disaster_df,left_on=["STATEFIPS","FIPS_remove0"],right_on = ["fipsStateCode","fipsCountyCode"],how = "left")
combined = combined.rename(columns={"declarationType":"Count"})
combined

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,converted_gisjoin,FIPS_remove0,fipsStateCode,fipsCountyCode,Count
0,Alabama,G0100010,41.0,1,10,Autauga,NaN,0100010,1,1.0,1.0,23.0
1,Alabama,G0100030,41.0,1,30,Baldwin,NaN,0100030,3,1.0,3.0,37.0
2,Alabama,G0100050,41.0,1,50,Barbour,NaN,0100050,5,1.0,5.0,27.0
3,Alabama,G0100070,41.0,1,70,Bibb,NaN,0100070,7,1.0,7.0,19.0
4,Alabama,G0100090,41.0,1,90,Blount,NaN,0100090,9,1.0,9.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3253,Wyoming,G5500650,68.0,56,390,Teton,NaN,5600390,39,56.0,39.0,5.0
3254,Wyoming,G5500670,68.0,56,410,Uinta,NaN,5600410,41,56.0,41.0,5.0
3255,Wyoming,G5500690,68.0,56,430,Washakie,NaN,5600430,43,56.0,43.0,5.0
3256,Wyoming,G5500710,68.0,56,450,Weston,NaN,5600450,45,56.0,45.0,7.0


In [16]:
combined.isna().sum()

STATE                   0
GISJOIN                 0
STATEICP               92
STATEFIPS               0
COUNTYFIPS              0
ALLCOUNTIES             0
VARIABLE NAME        3258
converted_gisjoin       0
FIPS_remove0            0
fipsStateCode          50
fipsCountyCode         50
Count                  50
dtype: int64

### Some Records Cannot be Matched Perfectly. Manual Fillout.

In [17]:
def manual(gis,ctname):
    target = combined[(combined["GISJOIN"]==gis) & (combined["ALLCOUNTIES"]==ctname)]
    try:
        cont_name = target["ALLCOUNTIES"].item()
    except:
        display(target)
        
    statefips = int(target["STATEFIPS"].item())
    combined.loc[combined["GISJOIN"]==gis,"Count"] = 0
         
    disaster_filter = disaster[disaster["designatedArea"].apply(lambda x: cont_name in x)]
    disaster_filter = disaster_filter[disaster_filter["fipsStateCode"]==statefips]
    combined.loc[combined["GISJOIN"]==gis,"Count"]=disaster_filter.shape[0]

In [18]:
combined[combined["fipsStateCode"].isna()].head(5)

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,converted_gisjoin,FIPS_remove0,fipsStateCode,fipsCountyCode,Count
67,Alaska,G0100250,81.0,2,10,Aleutian Islands,NaN,0200010,1,NaN,NaN,NaN
70,Alaska,G0100310,81.0,2,20,Anchorage,NaN,0200020,2,NaN,NaN,NaN
71,Alaska,G0100330,81.0,2,40,Barrow,NaN,0200040,4,NaN,NaN,NaN
72,Alaska,G0100350,81.0,2,50,Bethel,NaN,0200050,5,NaN,NaN,NaN
73,Alaska,G0100370,81.0,2,60,Bristol Bay Borough,NaN,0200060,6,NaN,NaN,NaN


In [19]:
missing = combined.loc[combined["fipsStateCode"].isna(),["GISJOIN","ALLCOUNTIES"]].apply(lambda x: (x["GISJOIN"],x["ALLCOUNTIES"]),axis=1)
[manual(i[0],i[1]) for i in missing]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [20]:
combined[combined["fipsStateCode"].isna()].head(5)

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,converted_gisjoin,FIPS_remove0,fipsStateCode,fipsCountyCode,Count
67,Alaska,G0100250,81.0,2,10,Aleutian Islands,NaN,0200010,1,NaN,NaN,1.0
70,Alaska,G0100310,81.0,2,20,Anchorage,NaN,0200020,2,NaN,NaN,7.0
71,Alaska,G0100330,81.0,2,40,Barrow,NaN,0200040,4,NaN,NaN,0.0
72,Alaska,G0100350,81.0,2,50,Bethel,NaN,0200050,5,NaN,NaN,6.0
73,Alaska,G0100370,81.0,2,60,Bristol Bay Borough,NaN,0200060,6,NaN,NaN,0.0


In [21]:
combined = combined[["STATE","converted_gisjoin","STATEFIPS","COUNTYFIPS","ALLCOUNTIES","FIPS_remove0","Count"]]
combined.head(60)

,STATE,converted_gisjoin,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,FIPS_remove0,Count
0,Alabama,0100010,1,10,Autauga,1,23.0
1,Alabama,0100030,1,30,Baldwin,3,37.0
2,Alabama,0100050,1,50,Barbour,5,27.0
3,Alabama,0100070,1,70,Bibb,7,19.0
4,Alabama,0100090,1,90,Blount,9,24.0
5,Alabama,0100110,1,110,Bullock,11,23.0
6,Alabama,0100130,1,130,Butler,13,27.0
7,Alabama,0100150,1,150,Calhoun,15,22.0
8,Alabama,0100170,1,170,Chambers,17,0.0
9,Alabama,0100190,1,190,Cherokee,19,0.0


In [22]:
holc_df_counties = holc_df.groupby("COUNTYGISJOIN").agg({"GISJOIN":"count"}).reset_index(drop=False)
holc_df_counties.loc[holc_df_counties["COUNTYGISJOIN"]=="1200860","COUNTYGISJOIN"] = "1200086"

In [23]:
combined=combined.merge(holc_df_counties,left_on="converted_gisjoin",right_on="COUNTYGISJOIN",how="left")

In [24]:
combined[combined["STATE"].isna()]

,STATE,converted_gisjoin,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,FIPS_remove0,Count,COUNTYGISJOIN,GISJOIN


In [25]:
combined["EPSV1"] = ((combined["Count"]>np.mean(combined["Count"])) & (~combined["GISJOIN"].isna())).astype(int)
combined=combined[["STATEFIPS","EPSV1","ALLCOUNTIES"]]

In [26]:
county_list = pd.read_excel("..\Public Data\VARIABLE for coder X.xlsx")
county_list = county_list.drop(columns="VARIABLE NAME")
labeled = county_list.merge(combined,on=["STATEFIPS","ALLCOUNTIES"],how="left")

In [27]:
labeled.sum()

STATE          AlabamaAlabamaAlabamaAlabamaAlabamaAlabamaAlab...
GISJOIN        G0100010G0100030G0100050G0100070G0100090G01001...
STATEICP                                                132120.0
STATEFIPS                                                 101400
COUNTYFIPS                                               3242168
ALLCOUNTIES    AutaugaBaldwinBarbourBibbBlountBullockButlerCa...
EPSV1                                                        128
dtype: object

In [28]:
labeled.to_csv(".\Results\EPSV1.csv")